# Anylsis Pipeline for the project
Team Dog: Maximilian Bernhardt, Patrick Ebner, Simon Enkel, Claudius Hilser, Enno Schwenk

* **Beschreibung der Pipeline nach Fertigstellung**



### Imports

In [1]:
#Import packages
import numpy as np
import pandas as pd

# Import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_style("white")

### Data read-in

In [ ]:
# File paths
tsv_file = '.\\Participant_data/Patrick.tsv'
#log_file = '.\\Participant_data/subject-Claudius_log.txt'
#csv_file = '.\\Participant_data/subject-Claudius.csv'

# Load the data
data_tsv = pd.read_csv(tsv_file, sep='\t')
#data_csv = pd.read_csv(csv_file)

### Data Preprocessing
In the data preprocessing, the data gets prepared for the analysis following in this script. Therefore, the time gets normalized to start at 0 and the fixations are extracted.

In [ ]:
# Normalizing time to start at 0


# Extracting fixations


### Defining Functions
All functions used for the visualizing the results of the research are defined in the following blocks. 


In [ ]:
# function 1



# function 2



#...

### Data Vizualization
In the following parts, the data gets visualized using different types of plots. Target of these plots is to deal with the topic: "Does diagonal positioning of stimuli eliminate the left-bias in viewing images?​" 


In [ ]:
#Data Visualization